In [2]:
import numpy as np #lin. Alg.
import matplotlib.pyplot as plt
import matplotlib.image as matimg

#importing keras modules
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense, Input, Lambda, Add, Subtract
from keras.layers import Conv2D, Softmax, Reshape
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, ReLU
from keras.layers import Dropout
from keras.layers import Activation, BatchNormalization
from keras.models import load_model, Model
from keras import backend as K
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical


#Python Image Library for image processing
from PIL import Image

import os

#Path of parasite and uninfected folders
path="E:MachineLearning/DVHacks/images-and-features/"


Using TensorFlow backend.


In [3]:
#Clear GPU Memory of Keras Models
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    #print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    
reset_keras() 


In [4]:
f = None
z = None

for i in os.listdir(path):
    if(i[0:5]=='pred_'):
        if f is None:
            f = np.load(path+i).reshape(1,40)
            z = np.load(path+'z_vector'+i[5:9]+'.npy').reshape(1,512)
            print(z.shape)
        else:
            f = np.concatenate((f,np.load(path+i).reshape(1,40)))
            z = np.concatenate((z,np.load(path+'z_vector'+i[5:9]+'.npy').reshape(1,512)))
print(f.shape)
print(z.shape)


(1, 512)
(2001, 40)
(2001, 512)


In [5]:
input_shape = (40,)

def MyModel():
    x = Input(shape=input_shape)
    #y = Dense(128, activation = 'relu')(x)
    o = Dense(512)(x)
    o = BatchNormalization()(o)
    #o = Lambda(lambda o:o*8)(o)

    return Model(x, o)

model = MyModel()
epochs = 10
learning_rate = 0.0001
decay_rate = learning_rate / epochs
momentum = 0.9
batch_size = 32

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False, clipvalue=0.5)
model.compile(loss=keras.losses.mean_absolute_error,
              optimizer=Adam(0.002),
              metrics=['accuracy', 'mae'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               20992     
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
Total params: 23,040
Trainable params: 22,016
Non-trainable params: 1,024
_________________________________________________________________
None


In [6]:
epoch_num = 250

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(f,z,test_size = .2, random_state = 42)

model_log = model.fit(x_train, y_train, epochs=epoch_num, batch_size=10, verbose=1, validation_data=(x_test, y_test), shuffle=True)

Train on 1600 samples, validate on 401 samples
Epoch 1/250
1600/1600 [==============================] - 1s 598us/step - loss: 0.9397 - acc: 0.0038 - mean_absolute_error: 0.9397 - val_loss: 0.9078 - val_acc: 0.0100 - val_mean_absolute_error: 0.9078
Epoch 2/250
1600/1600 [==============================] - 1s 328us/step - loss: 0.8433 - acc: 0.0000e+00 - mean_absolute_error: 0.8433 - val_loss: 0.8588 - val_acc: 0.0000e+00 - val_mean_absolute_error: 0.8588
Epoch 3/250
1600/1600 [==============================] - 1s 318us/step - loss: 0.8184 - acc: 0.0031 - mean_absolute_error: 0.8184 - val_loss: 0.8339 - val_acc: 0.0050 - val_mean_absolute_error: 0.8339
Epoch 4/250
1600/1600 [==============================] - 1s 317us/step - loss: 0.8076 - acc: 0.0038 - mean_absolute_error: 0.8076 - val_loss: 0.8193 - val_acc: 0.0050 - val_mean_absolute_error: 0.8193
Epoch 5/250
1600/1600 [==============================] - 0s 309us/step - loss: 0.8010 - acc: 0.0031 - mean_absolute_error: 0.8010 - val_loss:

KeyboardInterrupt: 

In [ ]:
#Plot accuracy vs Epochs
plt.plot(model_log.history['acc'])
plt.plot(model_log.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [7]:
model.save_weights('human_feature_to_z.h5')

In [8]:
model.evaluate(x_test,y_test)

401/401 [==============================] - 0s 50us/step


[0.8021456734497946, 0.007481296758104738, 0.8021456734497946]

In [31]:
pred = model.predict(x_train)
z_fin = np.zeros((1,512))
for i in range (1600):
    ztmp = pred[i] * x_train[i][2]
    #print(x_train[i][0])
    z_fin = np.add(z_fin,ztmp)
z_fin = z_fin/1600
print(z_fin)
print(z_fin.shape)
np.save('z_test.npy',model.predict(x_train)[0].reshape(1,512))
np.save('stylegan2/asian_vec.npy', z_fin)
print(x_train[0])
print(y_train[0])

[[ 1.77025194e-02  8.81230790e-03  1.34246302e-02  1.51890255e-02
  -2.02293698e-04  5.77887305e-03 -7.85315179e-04 -5.54116272e-03
   1.09529287e-02  2.50836570e-03  1.47998907e-02 -1.64470181e-03
  -1.75492489e-02  2.84912326e-03  1.43585603e-02 -1.00506419e-02
  -5.81229434e-03  4.07101246e-03  1.24338182e-02 -1.44329540e-02
   1.15719380e-02  1.61654862e-02  1.17975130e-02 -7.17732321e-03
  -9.41442543e-03  5.05424458e-03 -8.02659435e-04  1.74884595e-03
   1.02892387e-02  4.20410728e-03  1.74535819e-02 -1.62508143e-03
  -1.44824856e-02  2.23812606e-03 -1.05011632e-02  6.56406086e-03
   9.60633454e-03  7.38554801e-04 -7.20864866e-03 -1.75497566e-02
   1.25218238e-02 -1.01538487e-02 -6.42756887e-03  2.44320631e-03
  -6.21524512e-04 -2.43219961e-03 -2.20604349e-02  1.33952821e-02
   5.23276072e-03 -1.05287642e-02 -2.71281698e-03  2.21995456e-03
   1.61205467e-02  1.67534785e-03  6.41732786e-03  1.07069003e-02
  -1.33594092e-04  1.72802447e-02  1.49041564e-04  8.79511492e-03
   2.60460